In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import time
%matplotlib inline

In [2]:
ftypes_train = dict(row_id=np.int32, x=np.float32, y=np.float32, accuracy=np.int32, time=np.int32, place_id=np.int64)
#ftypes_test = dict(row_id=np.int32, x=np.float32, y=np.float32, accuracy=np.int32, time=np.int32)
df_train = pd.read_csv("train.csv", dtype=ftypes_train)
#df_test = pd.read_csv("test.csv", dtype=ftypes_test)

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
row_id      int32
x           float32
y           float32
accuracy    int32
time        int32
place_id    int64
dtypes: float32(2), int32(3), int64(1)
memory usage: 777.5 MB


In [4]:
df_train.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [6]:
df = pd.DataFrame(df_train['place_id'].value_counts())

In [7]:
df.head()

,place_id
8772469670,1849
1623394281,1802
1308450003,1757
4823777529,1738
9586338177,1718


In [8]:
df.reset_index(level=0, inplace=True)

In [9]:
df.head()

,index,place_id
0,8772469670,1849
1,1623394281,1802
2,1308450003,1757
3,4823777529,1738
4,9586338177,1718


In [10]:
df.shape

(108390, 2)

In [11]:
df.columns = ['place_id','count']

In [12]:
df.head()

,place_id,count
0,8772469670,1849
1,1623394281,1802
2,1308450003,1757
3,4823777529,1738
4,9586338177,1718


In [13]:
#df.iloc[range(0,108390,2)].tail()
df['group'] = 1234
#df.iloc[range(0,108390,2)] = 

In [14]:
df1 = df.iloc[range(0,108390,2)]
df2 = df.iloc[range(1,108390,2)]
#df.iloc[range(1,108390,2)]['group'] = 1

In [15]:
df1['group'] = 0
df2['group'] = 1

/home/ds/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ds/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
df_new = df1.append(df2,ignore_index=False)

In [17]:
df_new.sort_index(inplace=True)

In [18]:
df_new.head()

,place_id,count,group
0,8772469670,1849,0
1,1623394281,1802,1
2,1308450003,1757,0
3,4823777529,1738,1
4,9586338177,1718,0


In [19]:
df_new.columns

Index([u'place_id', u'count', u'group'], dtype='object')

In [20]:
df_new.drop(['count'],axis=1,inplace=True)

In [21]:
df_new.head()

,place_id,group
0,8772469670,0
1,1623394281,1
2,1308450003,0
3,4823777529,1
4,9586338177,0


In [22]:
fin = pd.merge(df_train, df_new, on='place_id', how='outer')

In [23]:
fin.isnull().sum(axis=0).tolist()

[0, 0, 0, 0, 0, 0, 0]

In [24]:
fin.tail()

,row_id,x,y,accuracy,time,place_id,group
29118016,28668518,7.1178,4.0088,75,183223,4212392186,0
29118017,28806302,9.1548,9.3501,42,627337,9762494481,0
29118018,28807080,8.0323,5.7283,57,773128,3110194770,0
29118019,28837438,1.5100,3.7481,71,370813,8262263483,1
29118020,28880886,1.9683,7.9141,71,559763,8806200160,1


In [25]:
fin.sort_values(by='row_id',ascending=True,inplace=True)

In [28]:
fin.tail()

,row_id,x,y,accuracy,time,place_id,group
25849975,29118016,6.5133,1.1435,67,399740,8671361106,0
19235275,29118017,5.9186,4.4134,67,125480,9077887898,0
24243646,29118018,2.9993,6.3680,67,737758,2838334300,0
3002697,29118019,4.0637,8.0061,70,764975,1007355847,0
211441,29118020,7.4523,2.0871,17,102842,7028698129,1


In [30]:
fin.to_csv('train_with_place_group.csv',index=False)